In [1]:
from gazpacho import get, Soup
from slack_sdk import WebClient # pip install slack_sdk
from slack_sdk.errors import SlackApiError
from dotenv import load_dotenv, find_dotenv # pip install python-dotenv
import os

### Create a Slack app

- To get started, create a new Slack App on [api.slack.com](https://api.slack.com/).
  1. Type in your app name.
  2. Select the workspace you'd like to build your app on.

### Give your app permissions

[Scopes](https://api.slack.com/scopes) give your app permission to do things (for example, post messages) in your development workspace.

- Navigate to **OAuth & Permissions** on the sidebar to add scopes to your app
- Scroll down to the **Bot Token Scopes** section and click **Add an OAuth Scope**.

For now, we'll only use one scope.

- Add the [`chat:write` scope](https://api.slack.com/scopes/chat:write) to grant your app the permission to post messages in channels it's a member of.
- Add the [`im:write` scope](https://api.slack.com/scopes/im:write) to grant your app the permission to post messages in DMs.

🎉 You should briefly see a success banner.

### Install the app in your workspace

- Scroll up to the top of the **OAuth & Permissions** pages and click the green "Install App to Workspace" button.

Next you'll need to authorize the app for the Bot User permissions.

- Click the "Allow" button.

🏁 Finally copy and save your bot token. You'll need this to communicate with Slack's Platform.

### Invite to channel

⚠️ @ mention the bot name in the desired channel before trying to post with it!

In [2]:
load_dotenv(find_dotenv())

SLACK_API_TOKEN = os.environ.get("SLACK_API_TOKEN")
client = WebClient(token=SLACK_API_TOKEN)
channel = '#web-scraping'

In [3]:
try:
    response = client.chat_postMessage(channel=channel, text="Hello world!")
    assert response["message"]["text"] == "Hello world!"
except SlackApiError as e:
    # You will get a SlackApiError if "ok" is False
    assert e.response["ok"] is False
    assert e.response["error"]  # str like 'invalid_auth', 'channel_not_found'
    print(f"Got an error: {e.response['error']}")

In [4]:
def parse(book):
    name = book.find('h4').text
    price = float(book.find('p').text[1:].split(' ')[0])
    return name, price

def fetch_sale():
    url = 'https://scrape.world/books'
    html = get(url)
    soup = Soup(html)
    books_raw = soup.find('div', {'class': 'book-'})
    books = [parse(book) for book in books_raw]
    on_sale = [name for name, price in books if price == 0.99]
    return '\n'.join(on_sale)

In [5]:
fetch_sale()

'Orconomics'

In [6]:
def post_to_channel():
    on_sale = fetch_sale()
    if on_sale:
        text = f'These books are on sale:\n{on_sale}\n\nBuy them now!'
    else:
        text = 'Nothing on sale :('
    try:
        response = client.chat_postMessage(channel='#web-scraping', text=text)
    except SlackApiError as e:
        assert e.response["ok"] is False
        assert e.response["error"]  # str like 'invalid_auth', 'channel_not_found'
        print(f"Got an error: {e.response['error']}")

In [7]:
post_to_channel()

In [8]:
import schedule # pip install schedule

In [9]:
while True:
    schedule.every(60).seconds.do(post_to_channel)
    # schedule.every().day.at('8:00').do(post_to_channel)

KeyboardInterrupt: 